# Run ALL imagenet networks with defaut param optimizer WITHOUT data augmentation

# HYPERPARAMETERS

### Test rmsprop vs SGD

### batch size 32 vs 64

### should show terrible variance

In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras.applications.vgg16 import decode_predictions
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time

import pandas

import pickle as pk

from keras.wrappers.scikit_learn import KerasClassifier

seed = 7

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


``` https://www.kaggle.com/kmader/transfer-learning-with-inceptionv3```

In [2]:
np.random.seed(seed)

In [3]:
data_dir = '/data/oxford102/train'
img_width, img_height = 256, 256
batch_size = 32
epochs = 100
nr_categories = 102

In [4]:
img_width, img_height = 256, 256
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
nb_train_samples = 4604
nb_validation_samples = 1094 

In [ ]:
img = load_img('/data/oxford102/jpg/image_00001.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='flower', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [ ]:
category = "/data/preview/"
imgs = np.random.choice(os.listdir(category), size=20, replace=False)
#os.listdir(category)

w=10
h=10
fig=plt.figure(figsize=(12, 12))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    #img = np.random.randint(10, size=(h,w))
    fig.add_subplot(rows, columns, i)
    img=mpimg.imread(category + imgs[i-1])
    print(img.shape)
    imgplot = plt.imshow(img)
plt.show()

In [36]:
def get_image_generator(input_processor, augmentation=False):
    if augmentation:
        return ImageDataGenerator(#rescale = 1./255, 
                                            rotation_range=40,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            shear_range=0.2,
                                            zoom_range=0.2,
                                            horizontal_flip=True,
                                            fill_mode='nearest',
                                            preprocessing_function=input_processor, 
                                            validation_split=0.2)
    else:
        return  ImageDataGenerator(preprocessing_function=input_processor, 
                                           validation_split=0.2)

In [6]:
def get_image_generators(network="vgg16", augmentation=False, input_shape=(256, 256)):
    input_processors = {
        "vgg16": keras.applications.vgg16.preprocess_input
    }
    
    train_val_datagen = get_image_generator(input_processors[network])
    
    train_generator = train_val_datagen.flow_from_directory(
            train_data_dir,  # this is the target directory
            target_size=(img_width, img_height),  # all images will be resized to 250x250
            batch_size=batch_size,
            subset="training",
            class_mode='categorical')
    validation_generator = train_val_datagen.flow_from_directory(
            train_data_dir,  # this is the target directory
            target_size=(img_width, img_height),  # all images will be resized to 250x250
            subset="validation",
            batch_size=batch_size,
            class_mode='categorical')
    
    return train_generator, validation_generator

In [7]:
train_generator, validation_generator = get_image_generators()

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.


In [8]:
img_width, img_height = 256, 256

In [ ]:
    models = {
        "vgg16": (keras.applications.vgg16.VGG16, keras.applications.vgg16.preprocess_input),
        "vgg19": (keras.applications.vgg19.VGG19, keras.applications.vgg19.preprocess_input),
        "inceptionv3": (applications.inception_v3.InceptionV3, keras.applications.inception_v3.preprocess_input),
        "xception": (applications.xception.Xception, keras.applications.xception.preprocess_input),
        "resnet50": (keras.applications.resnet50.ResNet50, keras.applications.resnet50.preprocess_input),
        "inceptionresnetv2": (keras.applications.inception_resnet_v2, keras.applications.inception_resnet_v2.preprocess_input),
        "mobilenet": (keras.applications.mobilenet.MobileNet, keras.applications.mobilenet.preprocess_input),
        "mobilenetv2": (keras.applications.mobilenet_v2, keras.applications.mobilenet_v2.preprocess_input),
        "densenet121": (keras.applications.densenet.DenseNet121, keras.applications.densenet.preprocess_input),
        "densenet169": (keras.applications.densenet.DenseNet169, keras.applications.densenet.preprocess_input),
        "densenet201": (keras.applications.densenet.DenseNet201, keras.applications.densenet.preprocess_input),
        "nastnetlarge": (keras.applications.nasnet.NASNetLarge, keras.applications.nasnet.preprocess_input),
        "nastnetmobile": (keras.applications.nasnet.NASNetMobile, keras.applications.nasnet.preprocess_input)} 


In [21]:
def build_model(network_name):
    k.set_learning_phase(0)
    
    if network_name == "vgg16":
        base_model = keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
    elif network_name == "vgg19":
        base_model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
    else:
        raise Exception("check your network name")
        
    for layer in base_model.layers[:]:
        layer.trainable = False
        
        #Adding custom Layers 
    k.set_learning_phase(1)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x, training=False)
    #x = Dense(102, activation="relu")(x)
    predictions = Dense(nr_categories, activation="softmax")(x)
    
    _model = Model(input = base_model.input, output = predictions)
    
    return _model
    

In [10]:
# for layer in base_model.layers[:]:
#     layer.trainable = False

In [11]:
# #Adding custom Layers 
# k.set_learning_phase(1)
# x = base_model.output
# x = Flatten()(x)
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = BatchNormalization()(x, training=False)
# #x = Dense(102, activation="relu")(x)
# predictions = Dense(nr_categories, activation="softmax")(x)

In [12]:
# model = Model(input = base_model.input, output = predictions)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [13]:
#train_generator, validation_generator = get_image_generators()

In [14]:
# compile the model 
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
#model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])
#model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0), metrics=["accuracy"])
#model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0), metrics=["accuracy"])
#model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [39]:
import os

def run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    np.random.seed(seed)
    log_time = time()
    params['log_time'] = log_time
    batch_size = params.get("batch_size")
    
    base = '/data/oxford102/experiments'
    path = os.path.join(base, str(log_time))
    checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
    tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

    try:
        if not os.path.exists(path):
            os.makedirs(path)
        history_callback = _model.fit_generator(
                train_generator,
                steps_per_epoch=num_train_img // batch_size,
                epochs=params.get("epochs"),
                validation_data=validation_generator,
                validation_steps=num_val_img // batch_size,
                callbacks = [checkpoint, early, tensorboard, csv_logger])
    except Exception as e:
        raise(e)
    finally:
        pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
        _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
        print(params)
        params

In [37]:
def compile_model(_model, optimizer):
    _model.compile(loss = "categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])


In [25]:
def get_optimizer(optimizer, params=None):

    if optimizer.lower() == "sgd":
        _params = params or dict(lr=0.0001, momentum=0.9)
        return optimizers.SGD(**_params)
    
    elif optimizer.lower() == "adam":
        _params = params or dict(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        return optimizers.Adam(**_params)
    
    elif optimizer.lower() == "adadelta":
        _params = params or dict(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
        return optimizers.Adadelta(**_params)
    
    elif optimizer.lower() == "adagrad":
        _params = params or dict(lr=0.01, epsilon=None, decay=0.0)
        return optimizers.Adagrad(**_params)
    
    elif optimizer.lower() == "rmsprop":
        _params = params or dict(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
        return optimizers.RMSprop(**_params)

In [38]:
from keras.initializers import glorot_uniform  # Or your initializer of choice
def run(params):
#     import tensorflow as tf
#     import gc
#     sess = tf.Session()

#     from keras import backend as K
#     K.set_session(sess)
#     model = None

#     gc.collect()
    
    __model = build_model(params["network_name"])
    
    opt = get_optimizer(params["optimizer"], params.get("optimizer_params"))
    print(opt)
    compile_model(__model, opt)
    
#     initial_weights = model.get_weights()
#     with sess.as_default():
#       new_weights = [glorot_uniform()(w.shape).eval() for w in initial_weights]
#     model.set_weights(new_weights)
    
    train_generator, validation_generator = get_image_generators()

    
    

    run_training(params.get("network_name"), __model, train_generator, validation_generator, params, 4604, 1094)
    
    

In [17]:
params = {'network_name': 'vgg16',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}

In [18]:
run(params)

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
71/71 [==============================] - 41s 582ms/step - loss: 14.5700 - acc: 0.0563 - val_loss: 13.8963 - val_acc: 0.1048

Epoch 00001: val_acc improved from -inf to 0.10478, saving model to /data/oxford102/experiments/1542065107.884702/vgg16_1542065107.884702.h5
Epoch 2/100
71/71 [==============================] - 34s 478ms/step - loss: 13.0808 - acc: 0.1426 - val_loss: 12.6350 - val_acc: 0.1783

Epoch 00002: val_acc improved from 0.10478 to 0.17831, saving model to /data/oxford102/experiments/1542065107.884702/vgg16_1542065107.884702.h5
Epoch 3/100
71/71 [==============================] - 35s 491ms/step - loss: 11.7604 - acc: 0.2345 - val_loss: 11.8201 - val_acc: 0.2151

Epoch 00003: val_acc improved from 0.17831 to 0.21507, saving model to /data/oxford102/experiments/1542065107.884702/vgg16_1542065107.884702.h5
Epoch 4/100
71/71 [==============================] - 34s 473ms/step - lo

In [32]:
run({'network_name': 'vgg16',
         'image_aug': False,
         'optimizer': 'rmsprop',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None})

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
71/71 [==============================] - 35s 496ms/step - loss: 1.1921e-07 - acc: 0.0062 - val_loss: 1.1921e-07 - val_acc: 0.0037

Epoch 00001: val_acc improved from -inf to 0.00368, saving model to /data/oxford102/experiments/1542067020.9744284/vgg16_1542067020.9744284.h5
Epoch 2/100
71/71 [==============================] - 34s 484ms/step - loss: 1.1921e-07 - acc: 0.0048 - val_loss: 1.1921e-07 - val_acc: 0.0037

Epoch 00002: val_acc did not improve from 0.00368
Epoch 3/100
71/71 [==============================] - 35s 495ms/step - loss: 1.1921e-07 - acc: 0.0045 - val_loss: 1.1921e-07 - val_acc: 0.0037

Epoch 00003: val_acc did not improve from 0.00368
Epoch 4/100
71/71 [==============================] - 34s 484ms/step - loss: 1.1921e-07 - acc: 0.0066 - val_loss: 1.1921e-07 - val_acc: 0.0037

Epoch 00004: val_acc did not improve from 0.00368
Epoch 00004: early stopping
{'network_name': 'v

In [33]:
run({'network_name': 'vgg16',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None})

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
143/143 [==============================] - 70s 487ms/step - loss: 1.1921e-07 - acc: 0.0055 - val_loss: 1.1921e-07 - val_acc: 0.0055

Epoch 00001: val_acc improved from -inf to 0.00551, saving model to /data/oxford102/experiments/1542067170.5056026/vgg16_1542067170.5056026.h5
Epoch 2/100
143/143 [==============================] - 69s 480ms/step - loss: 1.1921e-07 - acc: 0.0053 - val_loss: 1.1921e-07 - val_acc: 0.0055

Epoch 00002: val_acc did not improve from 0.00551
Epoch 3/100
143/143 [==============================] - 69s 485ms/step - loss: 1.1921e-07 - acc: 0.0055 - val_loss: 1.1921e-07 - val_acc: 0.0055

Epoch 00003: val_acc did not improve from 0.00551
Epoch 4/100
143/143 [==============================] - 69s 483ms/step - loss: 1.1921e-07 - acc: 0.0052 - val_loss: 1.1921e-07 - val_acc: 0.0055

Epoch 00004: val_acc did not improve from 0.00551
Epoch 00004: early stopping
{'network_n

In [34]:
run({'network_name': 'vgg16',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None})

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
35/35 [==============================] - 18s 507ms/step - loss: 1.1921e-07 - acc: 0.0054 - val_loss: 1.1921e-07 - val_acc: 0.0117

Epoch 00001: val_acc improved from -inf to 0.01172, saving model to /data/oxford102/experiments/1542067458.2843618/vgg16_1542067458.2843618.h5
Epoch 2/100
35/35 [==============================] - 17s 477ms/step - loss: 1.1921e-07 - acc: 0.0080 - val_loss: 1.1921e-07 - val_acc: 0.0117

Epoch 00002: val_acc did not improve from 0.01172
Epoch 3/100
35/35 [==============================] - 17s 480ms/step - loss: 1.1921e-07 - acc: 0.0063 - val_loss: 1.1921e-07 - val_acc: 0.0117

Epoch 00003: val_acc did not improve from 0.01172
Epoch 4/100
35/35 [==============================] - 17s 473ms/step - loss: 1.1921e-07 - acc: 0.0027 - val_loss: 1.1921e-07 - val_acc: 0.0117

Epoch 00004: val_acc did not improve from 0.01172
Epoch 00004: early stopping
{'network_name': 'v

In [35]:
run({'network_name': 'vgg16',
         'image_aug': False,
         'optimizer': 'rmsprop',
          'optimizer_params': None, 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None})

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
143/143 [==============================] - 70s 491ms/step - loss: 1.1921e-07 - acc: 0.0055 - val_loss: 1.1921e-07 - val_acc: 0.0055

Epoch 00001: val_acc improved from -inf to 0.00551, saving model to /data/oxford102/experiments/1542067538.452063/vgg16_1542067538.452063.h5
Epoch 2/100
143/143 [==============================] - 70s 489ms/step - loss: 1.1921e-07 - acc: 0.0055 - val_loss: 1.1921e-07 - val_acc: 0.0055

Epoch 00002: val_acc did not improve from 0.00551
Epoch 3/100
143/143 [==============================] - 69s 484ms/step - loss: 1.1921e-07 - acc: 0.0055 - val_loss: 1.1921e-07 - val_acc: 0.0055

Epoch 00003: val_acc did not improve from 0.00551
Epoch 4/100
143/143 [==============================] - 69s 485ms/step - loss: 1.1921e-07 - acc: 0.0055 - val_loss: 1.1921e-07 - val_acc: 0.0055

Epoch 00004: val_acc did not improve from 0.00551
Epoch 00004: early stopping
{'network_nam

# Train small convnet

In [ ]:
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPooling2D
# from keras.layers import Activation, Dropout, Flatten, Dense

# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(102, activation="softmax"))
# #model.add(Activation('sigmoid'))


# # #Adding custom Layers 
# # x = model.output
# # x = Flatten()(x)
# # x = Dense(1024, activation="relu")(x)
# # x = Dropout(0.5)(x)
# # #x = Dense(1024, activation="relu")(x)
# # predictions = Dense(nr_categories, activation="softmax")(x)



# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop', 
#               #optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),
#               metrics=['accuracy'])

# #model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


In [ ]:
model.summary()

In [ ]:
def get_model(optimizer=optimizers.SGD, verbose=False, image_size=(256,256)):
   
    img_width, img_height = input_shape

    params = dict(weights="imagenet",
                  include_top=False,
                  input_shape=(img_width, img_height, 3))
            
    models = {
        "vgg16": (keras.applications.vgg16.VGG16, keras.applications.vgg16.preprocess_input),
        "vgg19": (keras.applications.vgg19.VGG19, keras.applications.vgg19.preprocess_input),
        "inceptionv3": (applications.inception_v3.InceptionV3, keras.applications.inception_v3.preprocess_input),
        "xception": (applications.xception.Xception, keras.applications.xception.preprocess_input),
        "resnet50": (keras.applications.resnet50.ResNet50, keras.applications.resnet50.preprocess_input),
        "inceptionresnetv2": (keras.applications.inception_resnet_v2, keras.applications.inception_resnet_v2.preprocess_input),
        "mobilenet": (keras.applications.mobilenet.MobileNet, keras.applications.mobilenet.preprocess_input),
        "mobilenetv2": (keras.applications.mobilenet_v2, keras.applications.mobilenet_v2.preprocess_input),
        "densenet121": (keras.applications.densenet.DenseNet121, keras.applications.densenet.preprocess_input),
        "densenet169": (keras.applications.densenet.DenseNet169, keras.applications.densenet.preprocess_input),
        "densenet201": (keras.applications.densenet.DenseNet201, keras.applications.densenet.preprocess_input),
        "nastnetlarge": (keras.applications.nasnet.NASNetLarge, keras.applications.nasnet.preprocess_input),
        "nastnetmobile": (keras.applications.nasnet.NASNetMobile, keras.applications.nasnet.preprocess_input)} 

    _model, preprocess_input = models.get(model_name)[0](**params), models.get(model_name)[1]
    
    if verbose:
        _model.summary()

    return _model 

In [ ]:
def get_image_generator(image_augmentation=False, validation_split=0.2, preprocess_input=None):

    if not image_augmentation:
        return ImageDataGenerator(#rescale=1./255,
                                  preprocessing_function=preprocess_input,
                                  validation_split=validation_split)

    return ImageDataGenerator(
        #rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input,
        validation_split=validation_split)

In [ ]:
# train_val_datagen_no_aug = get_image_generator()

In [ ]:
# train_val_datagen = get_image_generator(True)

In [ ]:
# test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# train_generator = train_datagen.flow_from_directory(
#     data_dir,  # this is the target directory
#     target_size=(img_width, img_height),  # all images will be resized to 250x250
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset = "training")

In [ ]:
# validation_generator = train_datagen.flow_from_directory(
#     data_dir,  # this is the target directory
#     target_size=(img_width, img_height),  # all images will be resized to 250x250
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset="validation")

In [ ]:
def setup_model(params):
    data_dir = '/data/oxford102/train'
    img_width, img_height = params.get('image_size')
    batch_size =  params.get("batch_size")

    if type(params.get('optimizer')) in [keras.optimizers.Adam, keras.optimizers.SGD, keras.optimizers.RMSprop]:
        opt = params.get('optimizer')
    elif params.get("optimizer") == "rmsprop":
        opt = "rmsprop"
    elif params.get("optimizer") == "sgd":
        opt = optimizers.SGD(lr=0.0001, momentum=0.9)
    elif params.get("optimizer") == "adam":
        opt = optimizers.Adam()    
        
    model = None
    model = get_model(optimizer=opt, image_size=params.get('image_size'), n_categories=102, verbose=True)
       
    return model
    

In [ ]:
def get_generators(params):
    img_width, img_height = params.get('image_size')
    batch_size =  params.get("batch_size")
    
    if not params.get("image_aug"):
        train_val_datagen = get_image_generator()
    else: 
        train_val_datagen = get_image_generator(True)

    train_generator = train_val_datagen.flow_from_directory(
        data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        class_mode='categorical',
        subset = "training")    
        
    validation_generator = train_val_datagen.flow_from_directory(
        data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        class_mode='categorical',
        subset="validation")
    
    return train_generator, validation_generator

In [ ]:
import os

def run_training(model_name, model, train_generator, validation_generator, params, num_train_img, num_val_img):
    np.random.seed(seed)
    log_time = time()
    params['log_time'] = log_time
    batch_size = params.get("batch_size")
    
    base = '/data/oxford102/experiments'
    path = os.path.join(base, str(log_time))
    checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
    tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(log_time)), append=True, separator=';')

    try:
        if not os.path.exists(path):
            os.makedirs(path)
        history_callback = model.fit_generator(
                train_generator,
                steps_per_epoch=num_train_img // batch_size,
                epochs=params.get("epochs"),
                validation_data=validation_generator,
                validation_steps=num_val_img // batch_size,
                callbacks = [checkpoint, early, tensorboard, csv_logger])
    except Exception as e:
        raise(e)
    finally:
        pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
        model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
        print(params)
        params
        #with open("experiments")
        # Saving the Loss:
        # with the csvlogger callback
        # see the training callba balls

        # with pandas
        #pandas.DataFrame(history_callback.history).to_csv("history_small_convnet_rmsprop_50_100.csv")

        #with numpy
        # loss_history = history_callback.history["loss"]
        # numpy_loss_history = np.array(loss_history)
        # np.savetxt("loss_history_small_convnet_rmsprop_50_100.txt", numpy_loss_history, delimiter=",")


In [ ]:
from keras.initializers import glorot_uniform  # Or your initializer of choice
def run(params):
    import tensorflow as tf
    sess = tf.Session()

    from keras import backend as K
    K.set_session(sess)
    model = None
    import gc
    gc.collect()
    model = setup_model(params)
    initial_weights = model.get_weights()
    with sess.as_default():
      new_weights = [glorot_uniform()(w.shape).eval() for w in initial_weights]
    model.set_weights(new_weights)
    train_generator, validation_generator = get_generators(params)

    run_training(params.get("network_name"), model, train_generator, validation_generator, params, 4604, 1094)

params = {'network_name': 'vgg16'
         'image_aug': False,
         'optimizer': 'SGD', 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}

In [ ]:
from keras.initializers import glorot_uniform  # Or your initializer of choice
def run(params):
    import tensorflow as tf
    sess = tf.Session()

    from keras import backend as K
    K.set_session(sess)
    model = None
    import gc
    gc.collect()
    model = setup_model(params)
    initial_weights = model.get_weights()
    with sess.as_default():
      new_weights = [glorot_uniform()(w.shape).eval() for w in initial_weights]
    model.set_weights(new_weights)
    train_generator, validation_generator = get_generators(params)
    run_training(model, train_generator, validation_generator, params, 4604, 1094)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input


In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(259, 259, 3))

In [ ]:
base_model = InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))


In [ ]:
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# # let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# # and a logistic layer -- let's say we have 200 classes
# predictions = Dense(102, activation='softmax')(x)

In [ ]:
base_model.summary()

In [ ]:
"""
Both Flatten and GlobalAveragePooling2D are valid options. So is GlobalMaxPooling2D.

Flatten will result in a larger Dense layer afterwards, which is more expensive and may result in worse overfitting. But if you have lots of data, it might also perform better.

As usual, it depends completely on your problem.
"""

# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(102, activation="softmax"))

x = base_model.output
x = Flatten()(x)  # GlobalAveragePooling2D()(x) or GlobalMaxPooling2D()(x)
x = Dense(1024, activation='relu')(x)  #256
x = Dropout(0.5)(x)
predictions = Dense(102, activation='softmax')(x)



In [ ]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False



In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


In [ ]:
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
params = {'image_aug': True,
         'optimizer': 'SGD', 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}

In [ ]:
train_generator, validation_generator = get_generators(params)

In [ ]:
np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

num_train_img = 4604
num_val_img = 1094

base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "inception_resnetv2_{}.h5".format(log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "inception_resnetv2_{}.csv".format(log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    print(e)

In [ ]:

# Inception_v3 
np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

num_train_img = 4604
num_val_img = 1094

base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "inceptionv3_{}.h5".format(log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "inceptionv3_{}.csv".format(log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    print(e)

# FINE TUNE TOP LAYERS

In [ ]:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=["accuracy"])


In [ ]:
try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    print(e)

# TRAINING

In [ ]:
training_params = {
    1: {'log_time': None, 'optimizer': optimizers.rmsprop, 'image_augmentation': True, "tensorboard_id":1},
    2: {'log_time': None, 'optimizer': optimizers.rmsprop, 'image_augmentation': False, "tensorboard_id":2},
    2: {'log_time': None, 'optimizer': optimizers.SGD(lr=0.0001, momentum=0.9), 'image_augmentation': True, "tensorboard_id":3},
    2: {'log_time': None, 'optimizer': optimizers.SGD(lr=0.0001, momentum=0.9), 'image_augmentation': False, "tensorboard_id":4},
}

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}

In [ ]:
model = setup_model(params)
train_generator, validation_generator = get_generators(params)

In [ ]:
run_training(model, train_generator, validation_generator, params, 4604, 1094)

In [ ]:
params['log_time'] = 1541280636.0738575
params['optimizer'] = "rmsprop"

In [ ]:
params

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}

In [ ]:
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'sgd', 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}

In [ ]:
model = setup_model(params)
train_generator, validation_generator = get_generators(params)

In [ ]:
run_training(model, train_generator, validation_generator, params, 4604, 1094)

In [ ]:
params

In [ ]:
# params = {'image_aug': False,
#          'optimizer': 'sgd', 
#          'batch_size': 16,
#          'epochs': 100,
#          'image_size': (256, 256),
#          'log_time': None}

In [ ]:
params['optimizer'] = "adam"

In [ ]:
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'sgd', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}

In [ ]:
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'sgd', 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}

In [ ]:
run(params)

# Add dropout after each conv

In [ ]:
params = {'image_aug': False,
         'optimizer': 'sgd', 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

# DROPOUT AGAIN ONLY ON DENSE LAYER

In [ ]:
params = {'image_aug': False,
         'optimizer': 'sgd', 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'rmsprop', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'adam', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'adam', 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'adam', 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'adam', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'sgd', 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'sgd', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

# END

In [ ]:

#model.save_weights('first_try.h5')  # always save your weights after training or during training

In [ ]:
# Saving the Loss:
# with the csvlogger callback
# see the training callba balls

# with pandas
#pandas.DataFrame(history_callback.history).to_csv("history_small_convnet_rmsprop_50_100.csv")

#with numpy
# loss_history = history_callback.history["loss"]
# numpy_loss_history = np.array(loss_history)
# np.savetxt("loss_history_small_convnet_rmsprop_50_100.txt", numpy_loss_history, delimiter=",")

In [ ]:
# checkpoint = ModelCheckpoint("/data/oxford102/small_convnet_rmsprop_50_100.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
# tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)
# csv_logger = CSVLogger('small_convnet_1.csv', append=True, separator=';')


In [ ]:
# history_callback = model.fit_generator(
#         train_generator,
#         steps_per_epoch=4604 // batch_size,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=1094 // batch_size,
#         callbacks = [checkpoint, early, tensorboard, csv_logger])
# #model.save_weights('first_try.h5')  # always save your weights after training or during training

In [ ]:
# model.predict_classes()

In [ ]:
# no image aug

In [ ]:
# train_generator = train_datagen_no_aug.flow_from_directory(
#     data_dir,  # this is the target directory
#     target_size=(img_width, img_height),  # all images will be resized to 250x250
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset = "training")

# validation_generator = train_datagen_no_aug.flow_from_directory(
#     data_dir,  # this is the target directory
#     target_size=(img_width, img_height),  # all images will be resized to 250x250
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset="validation")

In [ ]:
# checkpoint = ModelCheckpoint("/data/oxford102/small_convnet_rmsprop_50_100_nodaug.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
# tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)
# csv_logger = CSVLogger('small_convnet_1.csv', append=True, separator=';')


In [ ]:
# history_callback = model.fit_generator(
#         train_generator,
#         steps_per_epoch=4604 // batch_size,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=1094 // batch_size,
#         callbacks = [checkpoint, early, tensorboard, csv_logger])
# #model.save_weights('first_try.h5')  # always save your weights after training or during training

In [ ]:
params = {'image_aug': True,
         'optimizer': 'rmsprop', 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'rmsprop', 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'rmsprop', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'rmsprop', 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
###

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'rmsprop', 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': "rmsprop", 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': "rmsprop", 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
###

In [ ]:
params = {'image_aug': False,
         'optimizer': 'sgd', 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'sgd', 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'sgd', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'sgd', 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
###

In [ ]:
params = {'image_aug': True,
         'optimizer': 'sgd', 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'sgd', 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'sgd', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'sgd', 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
###

In [ ]:
params = {'image_aug': True,
         'optimizer': 'adam', 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'adam', 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'adam', 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': 'adam', 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': 'adam', 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
## change LR, dropout, momentum

In [ ]:
params = {'image_aug': False,
         'optimizer': keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0), 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': keras.optimizers.RMSprop(lr=0.1, rho=0.9, epsilon=None, decay=0.0), 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': keras.optimizers.RMSprop(lr=0.001, rho=0.5, epsilon=None, decay=0.0), 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': keras.optimizers.RMSprop(lr=0.01, rho=0.5, epsilon=None, decay=0.0), 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': False,
         'optimizer': keras.optimizers.RMSprop(lr=0.1, rho=0.5, epsilon=None, decay=0.0), 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

If the loss increases and the accuracy increase too is because your regularization techniques are working well and you're fighting the overfitting problem. This is true only if the loss, then, starts to decrease whilst the accuracy continues to increase. Otherwise, if the loss keep growing your model is diverging and you should look for the cause (usually you're using a too high learning rate value).

### decrease lr

In [ ]:
params = {'image_aug': True,
         'optimizer': keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0), 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': keras.optimizers.RMSprop(lr=0.0001, rho=0.5, epsilon=None, decay=0.0), 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
# lets add decay, so we can get closer to the local minimum

In [ ]:
params = {'image_aug': True,
         'optimizer': keras.optimizers.RMSprop(lr=0.001, rho=0.5, epsilon=None, decay=0.1), 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
# batch size

In [ ]:
# Dropout (0.2 conv layers, 0.5 final)

In [ ]:
params = {'image_aug': True,
         'optimizer': "rmsprop",# keras.optimizers.RMSprop(lr=0.01, rho=0.5, epsilon=None, decay=0.0), 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
run(params)

In [ ]:
# image size

In [ ]:
params = {'image_aug': True,
         'optimizer': keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (128, 128),
         'log_time': None}
run(params)

In [ ]:
params = {'image_aug': True,
         'optimizer': keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (512, 512),
         'log_time': None}
run(params)

In [ ]:
def preprocess_img(img):

    image=mpimg.imread(img)
    imgplot = plt.imshow(image)

    image = load_img(img, target_size=(256, 256))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # do the same preprocessing as the vgg did
    image = preprocess_input(image)
    return image

In [ ]:
def predict(img_arr):
    # predict the probability across all output classes
    yhat = model.predict(img_arr)
    # convert the probabilities to class labels
    label = decode_predictions(yhat)
    # retrieve the most likely result, e.g. highest probability
    label = label[0][0]
    # print the classification
    print('%s (%.2f%%)' % (label[1], label[2]*100))

In [ ]:
img = '/data/oxford102/jpg/image_00001.jpg'
image_array = preprocess_img(img)
#predict(image_array)

In [ ]:
plot_model(model, to_file='baseline_CNN.png')

In [ ]:
pred = model.predict_proba(image_array)

In [ ]:
class_labels = ['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea', 'english marigold', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 'globe thistle', 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 'yellow iris', 'globe-flower', 'purple coneflower', 'peruvian lily', 'balloon flower', 'giant white arum lily', 'fire lily', 'pincushion flower', 'fritillary', 'red ginger', 'grape hyacinth', 'corn poppy', 'prince of wales feathers', 'stemless gentian', 'artichoke', 'sweet william', 'carnation', 'garden phlox', 'love in the mist', 'mexican aster', 'alpine sea holly', 'ruby-lipped cattleya', 'cape flower', 'great masterwort', 'siam tulip', 'lenten rose', 'barbeton daisy', 'daffodil', 'sword lily', 'poinsettia', 'bolero deep blue', 'wallflower', 'marigold', 'buttercup', 'oxeye daisy', 'common dandelion', 'petunia', 'wild pansy', 'primula', 'sunflower', 'pelargonium', 'bishop of llandaff', 'gaura', 'geranium', 'orange dahlia', 'pink-yellow dahlia?', 'cautleya spicata', 'japanese anemone', 'black-eyed susan', 'silverbush', 'californian poppy', 'osteospermum', 'spring crocus', 'bearded iris', 'windflower', 'tree poppy', 'gazania', 'azalea', 'water lily', 'rose', 'thorn apple', 'morning glory', 'passion flower', 'lotus', 'toad lily', 'anthurium', 'frangipani', 'clematis', 'hibiscus', 'columbine', 'desert-rose', 'tree mallow', 'magnolia', 'cyclamen ', 'watercress', 'canna lily', 'hippeastrum ', 'bee balm', 'ball moss', 'foxglove', 'bougainvillea', 'camellia', 'mallow', 'mexican petunia', 'bromelia', 'blanket flower', 'trumpet creeper', 'blackberry lily']

In [ ]:
_class_labels = np.array(class_labels)

In [ ]:
_class_labels

In [ ]:
for i,x in enumerate(pred[0]):
    if x == 1.0:
        print(_class_labels[i])

In [ ]:
probabilities = model.predict_generator(validation_generator)

In [ ]:
probabilities[1]> 0.5

# optimizer=SGD

In [ ]:
# checkpoint = ModelCheckpoint("/data/oxford102/small_convnet_sgd.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
# tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
# model.fit_generator(
#         train_generator,
#         steps_per_epoch=5698 // batch_size,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=2441 // batch_size,
#         callbacks = [checkpoint, early, tensorboard])
# #model.save_weights('first_try.h5')  # always save your weights after training or during training

# With Data augmentation

In [ ]:
# checkpoint = ModelCheckpoint("/data/oxford102/small_convnet_rmsprop_more_augmentation.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
# tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)
# model.fit_generator(
#         train_generator,
#         steps_per_epoch=5698 // batch_size,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=2441 // batch_size,
#         callbacks = [checkpoint, early, tensorboard])
# #model.save_weights('first_try.h5')  # always save your weights after training or during training

In [ ]:
# batch_size = 16

# # this is the augmentation configuration we will use for training
# train_datagen_no_aug = ImageDataGenerator(
#         rescale=1./255,
#         validation_split=0.2
# )
# train_datagen = ImageDataGenerator(
#         rotation_range=40,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest',
#         validation_split=0.2)

# # this is the augmentation configuration we will use for testing:
# # only rescaling
# test_datagen = ImageDataGenerator(rescale=1./255)

# # this is a generator that will read pictures found in
# # subfolers of 'data/train', and indefinitely generate
# # batches of augmented image data
# # train_generator = train_datagen.flow_from_directory(
# #         '/data/oxford102/train',  # this is the target directory
# #         target_size=(150, 150),  # all images will be resized to 150x150
# #         batch_size=batch_size,
# #         class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# # # this is a similar generator, for validation data
# # validation_generator = test_datagen.flow_from_directory(
# #         '/data/oxford102/val',
# #         target_size=(150, 150),
# #         batch_size=batch_size,
# #         class_mode='categorical')

# data_dir = '/data/oxford102/train'
# img_width, img_height = 256, 256
# batch_size = 32
# epochs = 100
    
# train_generator = train_datagen.flow_from_directory(
#     data_dir,  # this is the target directory
#     target_size=(img_width, img_height),  # all images will be resized to 250x250
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset = "training")

# validation_generator = train_datagen.flow_from_directory(
#     data_dir,  # this is the target directory
#     target_size=(img_width, img_height),  # all images will be resized to 250x250
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset="validation")